# Prédiction du score du nombre de buts marquer par chaque équipes lors d'un match

In [21]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import OneHotEncoder

In [22]:
games = pd.read_csv('archive/games.csv')
games = games.dropna()
games = games.drop_duplicates()
games = games.drop(['home_club_manager_name','away_club_manager_name', 'stadium', 'attendance', 'referee', 'url', 'home_club_name', 'away_club_name', 'aggregate', 'date'], axis=1)

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

In [24]:
games.head()

,game_id,competition_id,season,round,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,away_club_position,home_club_formation,away_club_formation,competition_type
0,2321027,L1,2013,1. Matchday,33,41,3,3,8.0,9.0,4-2-3-1,4-2-3-1,domestic_league
1,2321033,L1,2013,1. Matchday,23,86,0,1,13.0,7.0,4-3-2-1,4-3-1-2,domestic_league
2,2321044,L1,2013,2. Matchday,16,23,2,1,1.0,15.0,4-2-3-1,4-3-2-1,domestic_league
3,2321060,L1,2013,3. Matchday,23,24,0,2,18.0,11.0,4-3-2-1,4-2-3-1,domestic_league
4,2321072,L1,2013,5. Matchday,16,41,6,2,1.0,15.0,4-2-3-1,3-5-2,domestic_league


In [25]:
features = ['game_id', 'competition_id', 'season', 'round', 'home_club_id', 'away_club_id', 'home_club_position', 'away_club_position', 'home_club_formation', 'away_club_formation', 'competition_type']
target = ['home_club_goals', 'away_club_goals']

#vérifier si les colonnes sont présentes
missing_features = [feature for feature in features if feature not in games.columns]
if missing_features:
    raise ValueError(f"The following columns are missing from the dataframe: {missing_features}")

# Définir les variables catégorielles pour l'encodage
categorical_features = ['round', 'competition_id', 'home_club_formation', 'away_club_formation', 'competition_type']

# Définir les colonnes numériques
numerical_features = ['game_id', 'home_club_id', 'away_club_id', 'home_club_position', 'away_club_position',  'season']

In [27]:
# Créer un préprocesseur pour les colonnes numériques et catégorielles
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Créer un pipeline avec un modèle Random Forest
"""model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_estimators=100, random_state=0))
])"""

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1))  # Réduire n_estimators et utiliser n_jobs
])


# Vérifier si les colonnes cibles sont présentes
missing_target = [t for t in target if t not in games.columns]
if missing_target:
    raise ValueError(f"The following target columns are missing from the dataframe: {missing_target}")

# Diviser les données en ensembles d'entraînement et de test
X = games[features]
y = games[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Entraîner le modèle
model_pipeline.fit(X_train, y_train)

# Prédire les valeurs de test
y_pred = model_pipeline.predict(X_test)

# Évaluation du modèle
print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))

#Calcul du R2
r2 = r2_score(y_test, y_pred)
print("R2:", r2)

Mean Absolute Error: 0.9184701492537313
Mean Squared Error: 1.6722014925373134
R2: -0.08964178316258054


Bien que faible, la valeur négative du R-square indique que le modèle est moins performant qu’une prédiction basée sur une moyenne constante. Cela constitue une alerte signalant que le modèle n'est ni adapté ni performant.

Ce résultat est semblable à nos tentatives de création de modèles prédictifs pour **l'attribution d'un carton rouge ou jaune à un joueur lors d'un match**. Les résultats ne sont pas concluants, car il est évident que nous avons besoin de davantage de données. Par exemple, nous avons obtenu une \texttt{accuracy} améliorée de $+7%$ dans les prédictions de deux cartons rouges lors d'un match après que notre ensemble de données a été mis à jour. De plus, d'autres facteurs essentiels manquent pour affiner ce type de prédiction. **_Un carton_** jaune, en particulier, peut survenir à des moments inattendus dans un match en raison de facteurs très aléatoires, ce qui rend complexe l’élaboration d’un modèle fiable.

Une des pistes à explorer serait de revoir la valorisation des variables dans le dataset pour améliorer la classification lors de la création des modèles prédictifs. Par exemple, ajouter une classification des **_positions_**(positions tactiques:4-4-2 par exemple) des équipes, comme \texttt{offensives, défensives, offensive/défensive}, pourrait s’avérer utile. Ce type d’ajout ou de transformation pourrait améliorer nos modèles. Malheureusement, nous ne disposons pas d’assez de temps pour effectuer cette tâche dans le cadre de ce projet, mais cela constitue une piste de travail intéressante pour l’avenir.